**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo:**
  Gustavo Henrique de Andrade Pinheiro
**Matricula:**  2252822

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação: Gustavo Henrique de Andrade Pinheiro 2252822, ENGOC221N01

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?
Porque o algoritmo assume que, ao retirar o nó com menor distância estimada, esse valor já é definitivo. Arestas negativas poderiam reduzir distâncias de nós já finalizados, invalidando essa suposição.
2. Qual a complexidade do algoritmo com lista de adjacência e heapq?
Aproximadamente O((V + E) log V). Operações de heap dominam o custo; para grafos esparsos isso fica próximo de O(V log V).
💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [ ]:
import heapq
from typing import Dict, List, Tuple, Any


def dijkstra(grafo: Dict[Any, List[Tuple[Any, float]]], origem: Any):
"""Retorna dicionários: distancias (custo mínimo) e predecessores (pai).
grafo: dict[node] = [(vizinho, peso), ...]
origem: nó inicial
"""
# inicialização
dist = {no: float('inf') for no in grafo}
pai = {no: None for no in grafo}
dist[origem] = 0.0


fila = [(0.0, origem)] # (custo acumulado, nó)
vistos = set()


while fila:
custo_atual, nodo = heapq.heappop(fila)
if nodo in vistos:
continue
vistos.add(nodo)


for vizinho, peso in grafo[nodo]:
novo_custo = custo_atual + peso
if novo_custo < dist[vizinho]:
dist[vizinho] = novo_custo
pai[vizinho] = nodo
heapq.heappush(fila, (novo_custo, vizinho))


return dist, pai




def reconstruct(came_from: Dict[Any, Any], destino: Any) -> List[Any]:
"""Reconstrói o caminho usando o dicionário de predecessores.
Esta função é reutilizável tanto para Dijkstra quanto para A*.
"""
caminho = []
atual = destino
while atual is not None:
caminho.append(atual)
atual = came_from.get(atual)
caminho.reverse()
return caminho

In [ ]:
grafo_ex = {
'Hospital': [('A', 4), ('B', 2)],
'A': [('C', 3), ('D', 2)],
'B': [('A', 1), ('D', 7)],
'C': [('D', 4), ('Destino', 5)],
'D': [('Destino', 1)],
'Destino': []
}


distancias, pais = dijkstra(grafo_ex, 'Hospital')
print('Distâncias:', distancias)
print('Pais:', pais)
print('Caminho Hospital -> Destino:', reconstruct(pais, 'Destino'))

**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?
A geralmente expande menos nós que Dijkstra* quando a heurística é informativa, porque ela direciona a busca em direção ao objetivo.
2. Por que a heurística Manhattan é admissível nesse caso?
Manhattan é admissível em grids 4-direcionais com custo unitário: ela nunca superestima o número mínimo de passos.
💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [ ]:
import numpy as np
import heapq
import random
import matplotlib.pyplot as plt
from typing import Tuple, List


# heurística (Manhattan)
def heuristica(a: Tuple[int,int], b: Tuple[int,int]) -> int:
return abs(a[0]-b[0]) + abs(a[1]-b[1])




def gerar_grid(tam: int = 20, prob_obst: float = 0.15, seed: int = None) -> np.ndarray:
if seed is not None:
random.seed(seed)
np.random.seed(seed)
grid = np.zeros((tam, tam), dtype=int)
for i in range(tam):
for j in range(tam):
if random.random() < prob_obst:
grid[i, j] = 1
return grid




def vizinhos_4(pos: Tuple[int,int], grid: np.ndarray):
x, y = pos
passos = [(1,0), (-1,0), (0,1), (0,-1)]
for dx, dy in passos:
nx, ny = x + dx, y + dy
if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and grid[nx, ny] == 0:
yield (nx, ny)




def a_star(grid: np.ndarray, inicio: Tuple[int,int], objetivo: Tuple[int,int]):
"""Retorna (caminho, g_scores, came_from, expand_count)."""
aberto = [] # heap de (f, h, nó)
g_score = {inicio: 0}
came_from = {inicio: None}


f_inicio = heuristica(inicio, objetivo)
heapq.heappush(aberto, (f_inicio, heuristica(inicio, objetivo), inicio))


fechado = set()
expandidos = 0


while aberto:
f, h_val, atual = heapq.heappop(aberto)
if atual in fechado:
continue
fechado.add(atual)
expandidos += 1


if atual == objetivo:
caminho = reconstruct(came_from, objetivo)
return caminho, g_score, came_from, expandidos


for nb in vizinhos_4(atual, grid):
tentative_g = g_score[atual] + 1 # custo unitário por passo
if nb not in g_score or tentative_g < g_score[nb]:
g_score[nb] = tentative_g
came_from[nb] = atual
f_nb = tentative_g + heuristica(nb, objetivo)
heapq.heappush(aberto, (f_nb, heuristica(nb, objetivo), nb))


return None, g_score, came_from, expandidos

In [ ]:
grid = gerar_grid(tam=20, prob_obst=0.15, seed=42)
start = (0,0)
goal = (19,19)
# garantir início e fim livres
grid[start] = 0
grid[goal] = 0


caminho, gmap, came_from, exp = a_star(grid, start, goal)
print(f'Nós expandidos pelo A*: {exp}')
if caminho:
print(f'Passos do caminho: {len(caminho)-1}')
else:
print('Nenhum caminho encontrado')


# plot simples
plt.figure(figsize=(6,6))
plt.imshow(grid, cmap='gray_r', origin='upper')
if caminho:
xs = [c[1] for c in caminho]
ys = [c[0] for c in caminho]
plt.plot(xs, ys, linewidth=2)
plt.scatter([start[1], goal[1]], [start[0], goal[0]], c=['green','red'], s=40)
plt.gca().invert_yaxis()
plt.title('Grid e caminho encontrado (A*)')
plt.show()

**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?
Em-ordem num BST retorna os elementos ordenados — útil para listar produtos por preço.

Pré-ordem é prático para serializar ou clonar a árvore (visita raiz primeiro).

Pós-ordem é usado quando precisamos processar filhos antes do nó (ex.: liberar memória ou avaliar expressões).

In [ ]:
from dataclasses import dataclass
from typing import Optional, List


@dataclass
class No:
valor: int
esq: Optional['No'] = None
dir: Optional['No'] = None




def inserir_no(raiz: Optional[No], valor: int) -> No:
if raiz is None:
return No(valor)
if valor < raiz.valor:
raiz.esq = inserir_no(raiz.esq, valor)
else:
raiz.dir = inserir_no(raiz.dir, valor)
return raiz




def em_ordem(raiz: Optional[No], out: Optional[List[int]] = None) -> List[int]:
if out is None:
out = []
if raiz:
em_ordem(raiz.esq, out)
out.append(raiz.valor)
em_ordem(raiz.dir, out)
return out




def pre_ordem(raiz: Optional[No], out: Optional[List[int]] = None) -> List[int]:
if out is None:
out = []
if raiz:
out.append(raiz.valor)
pre_ordem(raiz.esq, out)
pre_ordem(raiz.dir, out)
return out




def pos_ordem(raiz: Optional[No], out: Optional[List[int]] = None) -> List[int]:
if out is None:
out = []
if raiz:
pos_ordem(raiz.esq, out)
pos_ordem(raiz.dir, out)
out.append(raiz.valor)
return out

In [ ]:
valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]
raiz = None
for v in valores:
raiz = inserir_no(raiz, v)


print('Em-ordem :', em_ordem(raiz))
print('Pré-ordem :', pre_ordem(raiz))
print('Pós-ordem :', pos_ordem(raiz))

**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?
Embora A* seja poderoso, não compensa quando: (1) a heurística é fraca (próxima de zero) — aí A* vira Dijkstra; (2) o custo de calcular a heurística é alto e anula o ganho; (3) o problema exige baixo uso de memória e A* consome muita memória com estruturas abertas; (4) em aplicações onde se aceita aproximação rápida em vez de otimalidade, algoritmos heurísticos mais simples podem ser preferíveis.
2. Diferencie corretude e otimalidade nos algoritmos estudados.
Corretude significa que o algoritmo produz uma solução válida quando há solução; otimalidade significa que ele produz a melhor solução (por exemplo, menor custo). Um algoritmo pode ser correto sem ser ótimo. Dijkstra e A* (com heurística admissível) são ambos ótimos sob as condições corretas.
3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.
Em-ordem: imprimir uma lista de preços do menor para o maior (vendedores/estoque).

Pré-ordem: salvar a estrutura de pastas/árvore para restaurar depois (serialização).

Pós-ordem: remover diretórios recursivamente (deletar arquivos filhos antes do diretório pai) ou avaliar expressões aritméticas.
4. Como heurísticas inconsistentes podem afetar o resultado do A*?
Heurísticas inconsistentes podem fazer com que nós precisem ser reabertos (re-expandidos) quando se encontra um caminho melhor, aumentando o custo da busca. Com implementação que suporta reabertura, a otimalidade continua garantida se a heurística for admissível; sem reabertura, uma heurística inconsistente pode levar à perda da otimalidade.
💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

Insira suas respostas aqui

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.